# Powersenosr入门教程2 - 常用通信接口

# Powersensor 排针管脚定义

Powersensor 支持串口（molex接口），spi接口，i2c接口等通信方式，位置图如下：

![img//ps_backInterface.png](./img/ps_backInterface.png)

In [1]:
import PowerSensor as ps
import numpy as np
import cv2
import matplotlib.pyplot as pyplot

# SPI 通信示例

Powersenosr支持的SPI是一类双向全双工的SPI通信协议，片选信号硬件管理。管脚有CS（片选）、sck（时钟）、MOSI（主机输出从机输入）、MISO（主机输入从机输出）4个。

示例程序如下：

In [2]:
# 初始化spi，运行一次即可
spi = ps.SpiPort()

In [5]:
# 配置spi，参数分别是通信速率，sck极性，sck激活沿
## sck极性0或1，0代表空闲状态低电平，1代表空闲状态高电平
## sck激活沿，0代表第一个沿数据位有效（MOSI和MISO)，1代表第二个沿数据为有效。
## 多说一句，0，0意味着空闲sck为低，上升沿读写数据，1，0意味着空闲sck为高，下降沿读写数据。
spi.set_clock(1000000, 0, 0)

In [6]:
# 读写数据，回环测试：请将mosi和miso短接，这样发送数据的时候就可以收到数据
# 读写的数据类型为list，dtype为uint8
to_send = [1, 3, 5, 7]
print(spi.swap_data(to_send))

[1, 3, 5, 7]


# I2C通信示例

i2c是一种双线多设备总线结构的通信协议，考虑到兼容性，在与外设通信时注意自己接合适的上拉电阻（powersensor没有接上拉电阻）。

i2c无法回环测试，需要外接一个设备，这里以mpu6050举例，示例程序如下：

In [13]:
# 初始化i2c，运行一次即可
i2c = ps.I2cPort()

In [14]:
# 配置从机地址，注意，这里使用的是7位的从机地址，
# mpu6050文档上写的从机地址是0xD0，这个是8位地址，需要右移一位变成0x68
i2c.set_slave_addr(0x68)

In [15]:
# 这里读取mpu6050的ID寄存器，这个寄存器是只读的，值为104
imu_id = i2c.read_byte(117)
print(imu_id)

104


# 串口通信

串口管脚是一个molex接口，这种接口非常稳定

![串口管脚](./img/ps_interfaceUsart.jpg)

从左往右1、2、3、4管脚依次为：

管脚号|颜色|名称
:-: | :-: | :-: 
1	| 红 |	TXD0 |
2	| 黑	| RXD0 |
3	| 黄	| GND |
4	| 绿	| 3V3 |

串口在测试时需要使用usb转串口模块（包装里有） 及串口调试助手，接线的时候注意tx接rx，rx接tx。

In [17]:
# 初始化串口，运行一次即可
s1 = ps.UsartPort()

## 字符串打印，上位机读取常用的方式

In [18]:
# 设置波特率，只需要设置一次，默认为115200
s1.set_baudrate(115200)
for i in range(2):
    s1.u_print('hello world!\n')

## 输出字节数组，单片机读取常用的方式

In [19]:
# 设置波特率，只需要设置一次，默认为115200
s1.set_baudrate(9600)
for i in range(2):
    s1.u_send_bytes([1, 3, 5, 7, 9])

结果如图所示：
![](./img/ps_usartResutl2.jpg/)

# PWM输出

powersensor目前支持两种PWM输出模式，一种是4路普通PWM，这四路pwm波的周期是一致的，高电平时间可以分别设定；另一种是2路互补PWM，每路PWM包含一个正相位信号和一个反相位信号。

In [4]:
# 初始化控制对象
pwmx = ps.PwmActuator()

In [6]:
# 普通模式下的pwm输出
# 预分频，实际分频数=设定值+1
pwmx.preDividor = 10 - 1
# 死区，实际的周期=设定值+1
pwmx.period = 1000 - 1
# 高电平时间，最大为周期的设定值，即999
pwmx.pwm0 = 200
pwmx.pwm1 = 400
pwmx.pwm2 = 600
pwmx.pwm3 = 800
# 模式设置，0：低电平；1：普通模式；2：互补输出模式；
pwmx.mode = 1
# 写入设置，写入这句后，上述配置的内容才起效
pwmx.pwm_setup(True)

In [7]:
# 互补模式下的pwm输出
# 预分频，实际分频数=设定值+1
pwmx.preDividor = 10 - 1
# 周期，实际的周期=设定值+1
pwmx.period = 1000 - 1
# 死区，即互补的两个信号的高电平间的间隔，实际值为设定值+1
pwmx.preDeadZonee = 50 - 1
# 互补滤波通道1，高电平时间，最大为周期的设定值，即999
pwmx.pwm0 = 200
# 互补滤波通道2，高电平时间，最大为周期的设定值，即999
pwmx.pwm2 = 600
## 此模式下，pwm1和pwm3不起作用
# 模式设置，0：低电平；1：普通模式；2：互补输出模式；
pwmx.mode = 1
# 写入设置，写入这句后，上述配置的内容才起效
pwmx.pwm_setup(True)
# 预分频，实际分频数=设定值+1
pwmx.preDividor = 10 - 1

In [6]:
# 动态改变高电平时间，第一个参数为通道数0-3，第二个参数为高电平时间
pwmx.pwm_set_width(0, 300)
